In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

import RVDLM

# ------------------------------------------------------
# 0. Inspect available submodules / functions
# ------------------------------------------------------
rvu = RVDLM.rvdlm_univariate
tune = RVDLM.tuning
dlm = getattr(RVDLM, "dlm_baseline", None)  # may be None if not imported in __init__

print("RVDLM.rvdlm_univariate symbols:")
print([name for name in dir(rvu) if not name.startswith("_")])

print("\nRVDLM.tuning symbols:")
print([name for name in dir(tune) if not name.startswith("_")])

if dlm is not None:
    print("\nRVDLM.dlm_baseline symbols:")
    print([name for name in dir(dlm) if not name.startswith("_")])
else:
    print("\nNo dlm_baseline module exposed via RVDLM.__init__ (baseline will be skipped).")

# ------------------------------------------------------
# 1. Download SPY daily data (2000–2025)
# ------------------------------------------------------
data = yf.download("SPY", start="2000-01-01", end="2025-01-01", interval="1d")

# Drop Ticker level if yfinance returns a MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel("Ticker")

# ------------------------------------------------------
# 2. Build log prices, RS realized variance, and precision
# ------------------------------------------------------
ohlc_log = np.log(data[["Open", "High", "Low", "Close"]]).copy()

# Rogers–Satchell realized variance (on log prices)
rs_var = (
    (ohlc_log["High"] - ohlc_log["Open"]) * (ohlc_log["High"] - ohlc_log["Close"])
    + (ohlc_log["Open"] - ohlc_log["Low"]) * (ohlc_log["High"] - ohlc_log["Low"])
)

df = pd.DataFrame(index=ohlc_log.index)
df["Close"] = ohlc_log["Close"]
df["Vol"] = rs_var
df["precision"] = 1.0 / df["Vol"]

# Your earlier code used the typo 'percision', so mirror it in case you reuse old functions later
df["percision"] = df["precision"]

# Clean up infinities / NaNs, and keep only positive precision
df = df.replace([np.inf, -np.inf], np.nan).dropna()
df = df[df["precision"] > 0]

print("\nHead of working DataFrame:")
print(df.head())

# ------------------------------------------------------
# 3. Priors for RV–DLM and classical DLM
# ------------------------------------------------------
# RV–DLM state: [intercept, lag1, (optionally) leverage term]
# Your current RVDLM_Univariate._design_vector uses:
#   - with use_leverage=False: [1, y_{t-1}]
#   - with use_leverage=True : [1, y_{t-1}, x_t]
# We'll start with use_leverage=False for simplicity (2-dim state).
use_leverage = False

if use_leverage:
    m0_rv = np.array([0.0, 0.9, 0.0])
    C0_rv = np.eye(3) * 0.01
else:
    m0_rv = np.array([0.0, 0.9])
    C0_rv = np.eye(2) * 0.01

# Gamma / dynamic-F variance block initialization
n0, s0 = 49.0, 1.0 / 5000.0

# Classical DLM priors: state [intercept, lag1]
m0_dlm = np.array([0.0, 0.9])
C0_dlm = np.eye(2) * 0.01
nu0, S0 = 2.0, 0.001

y = df["Close"].values

# ------------------------------------------------------
# 4. Tune the RV–DLM via tune_rvdlm_ohlc
# ------------------------------------------------------
best_rv_params, best_rv_nll = tune.tune_rvdlm_ohlc(
    df=df,
    m0=m0_rv,
    C0=C0_rv,
    n0=n0,
    s0=s0,
    beta_grid=None,    # use defaults inside tune_rvdlm_ohlc
    alpha_grid=None,
    lambda_grid=None,
    use_leverage=use_leverage,
)

beta_gamma, alpha_gamma, lambda_theta = best_rv_params

print("\nBest RV–DLM hyperparameters (from tune_rvdlm_ohlc):")
print(f"  beta_gamma   = {beta_gamma:.4f}")
print(f"  alpha_gamma  = {alpha_gamma:.4f}")
print(f"  lambda_theta = {lambda_theta:.4f}")
print(f"  Max log-lik  = {-best_rv_nll:.2f}")

# Build the best RV–DLM model explicitly and recompute log-likelihood
rv_model = rvu.RVDLM_Univariate(
    beta=beta_gamma,
    alpha=alpha_gamma,
    lambda_theta=lambda_theta,
    m0=m0_rv,
    C0=C0_rv,
    n0=n0,
    s0=s0,
    use_leverage=use_leverage,
)

rv_loglik = rv_model.loglik(df)
print(f"Recomputed RV–DLM log-likelihood: {rv_loglik:.2f}")

# ------------------------------------------------------
# 5. Tune the classical DLM baseline (if dlm_baseline is available)
# ------------------------------------------------------
if dlm is not None and hasattr(dlm, "grid_search_dlm"):
    best_dlm_params, best_dlm_nll = dlm.grid_search_dlm(
        y=y,
        m0=m0_dlm,
        C0=C0_dlm,
        nu0=nu0,
        S0=S0,
        lambda_theta_grid=None,   # use defaults inside grid_search_dlm
        lambda_sigma_grid=None,
    )
    best_lambda_theta, best_lambda_sigma = best_dlm_params

    print("\nBest classical DLM hyperparameters:")
    print(f"  lambda_theta = {best_lambda_theta:.4f}")
    print(f"  lambda_sigma = {best_lambda_sigma:.4f}")
    print(f"  Max log-lik  = {-best_dlm_nll:.2f}")

    # Recompute baseline log-likelihood using dlm_lag1_logpred
    ll_dlm = dlm.dlm_lag1_logpred(
        y=y,
        lambda_theta=best_lambda_theta,
        lambda_sigma=best_lambda_sigma,
        m0=m0_dlm,
        C0=C0_dlm,
        nu0=nu0,
        S0=S0,
    )
    dlm_loglik = np.sum(ll_dlm[1:])  # drop t=0

    print(f"Recomputed classical DLM log-likelihood: {dlm_loglik:.2f}")

    # ------------------------------------------------------
    # 6. Compare RV–DLM vs classical DLM (total and per-observation)
    # ------------------------------------------------------
    n_obs = len(y) - 1  # effective number of one-step forecasts
    delta_loglik = rv_loglik - dlm_loglik
    per_obs_delta = delta_loglik / n_obs
    per_obs_BF = np.exp(per_obs_delta)

    print("\nRV–DLM vs classical DLM comparison:")
    print(f"  Δ log-lik (RV–DLM - DLM)     = {delta_loglik:.2f}")
    print(f"  Per-observation Δ log-lik     = {per_obs_delta:.6f}")
    print(f"  Per-observation BF (exp(Δ))   = {per_obs_BF:.6f}")

else:
    print("\nNo classical DLM baseline available via RVDLM.dlm_baseline; "
          "skipping baseline comparison.")


/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_5807/3612793093.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("SPY", start="2000-01-01", end="2025-01-01", interval="1d")
[*********************100%***********************]  1 of 1 completed

RVDLM.rvdlm_univariate symbols:
['DynamicGammaFilter', 'RVDLM_Univariate', 'np', 'pd', 'student_t']

RVDLM.tuning symbols:
['RVDLM_Univariate', 'np', 'pd', 'tune_rvdlm_ohlc']

RVDLM.dlm_baseline symbols:
['dlm_lag1_logpred', 'grid_search_dlm', 'np', 'student_t']

Head of working DataFrame:
               Close       Vol     precision     percision
Date                                                      
2000-01-03  4.520569  0.000897   1114.439248   1114.439248
2000-01-04  4.480677  0.000969   1031.999491   1031.999491
2000-01-05  4.482464  0.000719   1391.132618   1391.132618
2000-01-06  4.466262  0.000721   1386.154441   1386.154441
2000-01-07  4.522715  0.000071  14087.750315  14087.750315



Best RV–DLM hyperparameters (from tune_rvdlm_ohlc):
  beta_gamma   = 0.7500
  alpha_gamma  = 2.0000
  lambda_theta = 0.9900
  Max log-lik  = 20555.15
Recomputed RV–DLM log-likelihood: 20555.15

Best classical DLM hyperparameters:
  lambda_theta = 0.9900
  lambda_sigma = 0.9000
  Max log-lik  = 20311.36
Recomputed classical DLM log-likelihood: 20311.36

RV–DLM vs classical DLM comparison:
  Δ log-lik (RV–DLM - DLM)     = 243.79
  Per-observation Δ log-lik     = 0.038784
  Per-observation BF (exp(Δ))   = 1.039546
